In [2]:
import pandas as pd
import numpy as np
import requests
import time
import pandas as pd
from sqlalchemy import create_engine
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

from config import weather_api_key

In [3]:
wine_file = "Resources/winemag-data-130k-v2.csv"
wine_df = pd.read_csv(wine_file)
wine_df

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [7]:
cleaned_df = wine_df[["variety", "points", "price", "country", "province", "region_1"]]
cleaned_df.dropna()
cleaned_df.loc[cleaned_df["country"]== "US"]


,variety,points,price,country,province,region_1
2,Pinot Gris,87,14.0,US,Oregon,Willamette Valley
3,Riesling,87,13.0,US,Michigan,Lake Michigan Shore
4,Pinot Noir,87,65.0,US,Oregon,Willamette Valley
10,Cabernet Sauvignon,87,19.0,US,California,Napa Valley
12,Cabernet Sauvignon,87,34.0,US,California,Alexander Valley
...,...,...,...,...,...,...
129945,Chenin Blanc,90,20.0,US,California,Santa Ynez Valley
129949,Chardonnay,90,35.0,US,California,Napa Valley
129950,Zinfandel,90,35.0,US,California,Napa Valley
129952,Zinfandel,90,22.0,US,California,Chiles Valley


In [5]:
province = cleaned_df[["country", "province"]]
province.drop_duplicates()

,country,province
0,Italy,Sicily & Sardinia
1,Portugal,Douro
2,US,Oregon
3,US,Michigan
5,Spain,Northern Spain
...,...,...
112132,Greece,Vin de Pays de Velvendo
118120,Germany,Landwein Rhein
121719,Greece,Lesbos
123510,Portugal,Távora-Varosa


In [8]:
longitude = []
latitude = []
   
# function to find the coordinate
# of a given city 
def findGeocode(city):
       
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="your_app_name")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
        
        return findGeocode(city) 

In [ ]:
for i in (province["province"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
          
        # coordinates returned from 
        # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    # if coordinate for a city not
    # found, insert "NaN" indicating 
    # missing value 
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [ ]:
# now add this column to dataframe
province["Longitude"] = longitude
province["Latitude"] = latitude
  
province

In [ ]:
lat = []
lng = []
max_temp = []
humid = []
cloudy = []
winds = []
country = []
date = []
city_weather = []

#variables for records
record_num = 0
Set = 1

print("Beginning Data Retrieval")
print("----------------------------")

#loop through cities retreiving data for each city
for i in (province["province"]):
    time.sleep(1)
    
    #check to see if there is data for city
    try:
        query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={i}&appid={weather_api_key}&units=imperial")
        response = requests.get(query_url).json()

        if record_num == 50:
            Set += 1
            record_num = 0
        else:
            record_num += 1 
            
        #print record saying which city is being processed
        print(f"Processing Record {record_num} of Set {Set} | {i}")
        
        #get data for city 
        cities_lng = response['coord']['lon']
        cities_lat = response['coord']["lat"]
        cities_max_temp = response['main']['temp_max']
        cities_humid = response['main']['humidity']
        cities_cloudy = response['clouds']['all']
        cities_winds = response['wind']['speed']
        cities_country = response['sys']['country']
        cities_date = response['dt']

        #Create dictionary with data gathered
        city_weather.append({
        "City": i,
        "Lat": cities_lat,
        "Lng": cities_lng,
        "Max Temp": cities_max_temp,
        "Humidity": cities_humid,
        "Cloudiness": cities_cloudy,
        "Wind Speed": cities_winds,
        "Country": cities_country,
        "Date": cities_date
        })

    except:
        print(f"City {i} not found... Sorry!")
        pass
    12
print("Complete!")